# 03_Plot_Walktrap_clustering_performance

cc_mamba_simba

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad
import seaborn as sns   

# approximate original figure in http://spatial.libd.org/spatialLIBD/
palette_celltype={'L1':'#eb34a8',
                  'L2':'#3486eb',
                  'L3':'#34eb5b',
                  'L4':"#ae34eb",
                  'L5':'#ebdb34',
                  'L6':'#eb9234',
                  'WM':'#000000'}

palette_entity_anno = palette_celltype.copy()
palette_entity_anno['gene'] = "lightgray"

import simba as si

/data/pinello/SHARED_SOFTWARE/anaconda_latest/envs/cc_envs/cc_mamba_simba/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import igraph

In [3]:
spatialpca_fn = '../results/00/spatialpca/spatialpca_adata_py.tsv'
spatialpca_df = pd.read_csv(spatialpca_fn, sep='\t', index_col=0)

In [6]:
adata = sc.read_h5ad(spatialpca_df['spatialPCA_adata_fn'].iloc[0])

In [9]:
adata.obsm['X_pca']

array([[ 1.83939471e+00, -5.61985677e-02,  1.19041991e-01, ...,
         1.66866199e+00, -6.65284314e-01,  4.67355109e-01],
       [-4.20172655e+00, -1.70724778e-01, -8.19068641e-01, ...,
         8.57655710e-02,  6.69085248e-02,  2.67064886e-01],
       [ 8.30068342e+00,  8.12987118e+00, -2.84636713e+00, ...,
         2.01794839e-03, -5.43444085e-01,  1.45271695e+00],
       ...,
       [ 1.02635875e+01, -7.83128169e+00,  1.19717575e+00, ...,
         1.27000713e+00, -1.12556297e+00,  1.40028195e+00],
       [-1.41694845e+00, -2.25815502e+00, -2.29966323e-01, ...,
        -5.38051584e-01, -9.48858327e-02, -8.50527983e-01],
       [ 4.78863486e+00,  6.89225962e+00, -1.68206667e+00, ...,
         6.90637438e-01,  3.42647802e-01,  1.67227343e-01]])

In [14]:
# from chatGPT
import numpy as np
from sklearn.neighbors import NearestNeighbors
import igraph as ig

def build_snn_graph(data, k=5):
    """
    Build a Shared Nearest Neighbor (SNN) graph from the data.

    Parameters:
        data (np.ndarray): Data points (n_samples, n_features).
        k (int): Number of nearest neighbors.

    Returns:
        ig.Graph: SNN graph with edge weights as shared neighbor counts.
    """
    # Step 1: Compute k-nearest neighbors
    nbrs = NearestNeighbors(n_neighbors=k+1, metric='euclidean').fit(data)
    distances, indices = nbrs.kneighbors(data)
    
    # Remove the point itself from its neighbors (first neighbor is always the point itself)
    knn_indices = indices[:, 1:]
    
    # Step 2: Build the SNN graph
    num_nodes = data.shape[0]
    edges = []
    weights = []

    for i in range(num_nodes):
        for j in knn_indices[i]:
            if i < j:  # Avoid duplicates since the graph is undirected
                # Shared neighbors between i and j
                shared_neighbors = len(set(knn_indices[i]) & set(knn_indices[j]))
                
                if shared_neighbors > 0:
                    edges.append((i, j))
                    weights.append(shared_neighbors)

    # Step 3: Create igraph object
    g = ig.Graph(edges=edges)
    g.es['weight'] = weights

    return g

def detect_communities_snn(data, k=5):
    """
    Detect communities using the SNN graph and Walktrap algorithm.

    Parameters:
        data (np.ndarray): Data points (n_samples, n_features).
        k (int): Number of nearest neighbors for SNN.

    Returns:
        list: Community membership for each node.
    """
    snn_graph = build_snn_graph(data, k=k)
    walktrap = snn_graph.community_walktrap(weights=snn_graph.es['weight'])
    clusters = walktrap.as_clustering()
    return clusters.membership, clusters, snn_graph


In [16]:

np.random.seed(42)
data = adata.obsm['X_pca'].copy()

k = 10  # Number of neighbors
membership, clusters, snn_graph = detect_communities_snn(data, k=k)

In [18]:
clusters

In [20]:
layout = snn_graph.layout("fr", grid=True)
ig.plot(clusters, layout=layout, bbox=(500, 500), vertex_label=range(len(membership)))

AttributeError: Plotting not available; please install pycairo or cairocffi